# Multiclass Classification of Cultural Items
**Homework 1 - Multilingual Natural Language Processing**

*By Joshua Edwin & Clemens Kubach*

Add general project description here at the end.

## Installs
Check that all required dependencies are installed as defined in `pyproject.toml`. Follow the `README.md` for more detailed instructions.

## Machine Setup

### Imports

In [1]:
import pandas as pd
from datasets import DatasetDict, Dataset
import logging

try:
    from google.colab import userdata  # type: ignore

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

logger = logging.getLogger(__name__)

### Huggingface Login and Loading Data
To access the dataset, there are three options available that will be tried in the following fallback-order:


1.   From HF via HF_TOKEN secret/envvar if set.
2.   From HF via inserting the HF token manually in the login dialog.
3.   From local `./train.csv` and `./valid.csv` files.

Afterwards the hf dataset instance and train, val dataframes can be accessed via `hf_dataset`, `df_train` and `df_val`.

In [2]:
from os import environ

from datasets import load_dataset
from huggingface_hub import login
from datasets.exceptions import DatasetNotFoundError
from huggingface_hub.errors import HfHubHTTPError


def extract_dev_subsets_from_hf_dataset(
    ds: DatasetDict,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    _df_train = pd.DataFrame(ds["train"])  # Silver-labeled training set
    _df_validation = pd.DataFrame(ds["validation"])  # Gold-labeled dev set
    return _df_train, _df_validation


def read_hf_token() -> str | None:
    if IN_COLAB:
        try:
            return userdata.get("HF_TOKEN")
        except KeyError:
            return None
    else:
        return environ.get("HF_TOKEN", None)


def do_blocking_hf_login():
    # run the login in a separate cell because login is non-blocking
    try:
        token = read_hf_token()
        login(token=token)
        if token is None:
            # block until logged-in
            input("Press enter of finish login!")
    except (HfHubHTTPError, DatasetNotFoundError):
        print(
            "Login via HF_TOKEN secret/envvar and via manual login widget failed "
            "or not authorized."
        )


do_blocking_hf_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
def load_train_val_data() -> tuple[DatasetDict, pd.DataFrame, pd.DataFrame]:
    try:
        _ds = load_dataset("sapienzanlp/nlp2025_hw1_cultural_dataset")
        _df_train, _df_val = extract_dev_subsets_from_hf_dataset(_ds)
        return _ds, _df_train, _df_val
    except (HfHubHTTPError, DatasetNotFoundError) as e:
        logger.error(
            f"Something went wrong during HF dataset access: {e}. "
            "Falling back to local files:"
        )
    try:
        train_df = pd.read_csv("train.csv")
        valid_df = pd.read_csv("valid.csv")

        train_dataset = Dataset.from_pandas(train_df)
        valid_dataset = Dataset.from_pandas(valid_df)

        # Create a DatasetDict
        dataset_dict = DatasetDict(
            {"train": train_dataset, "validation": valid_dataset}
        )
        return dataset_dict, train_df, valid_df
    except FileNotFoundError:
        raise FileNotFoundError(
            "Tried to access the dataset from Huggingface "
            "(via HF_TOKEN secret/envvar and manual auth) and from the local disk"
            "(via train.csv and valid.csv in the cwd) without success."
        )


hf_dataset, df_train, df_val = load_train_val_data()

# Show samples
print("\nTrain Set:")
display(df_train.head())

print("\nValidation Set:")
display(df_val.head())


Train Set:


,item,name,description,type,category,subcategory,label
0,http://www.wikidata.org/entity/Q32786,916,2012 film by M. Mohanan,entity,films,film,cultural exclusive
1,http://www.wikidata.org/entity/Q371,!!!,American dance-punk band from California,entity,music,musical group,cultural representative
2,http://www.wikidata.org/entity/Q3729947,¡Soborno!,Mort & Phil comic,entity,comics and anime,comics,cultural representative
3,http://www.wikidata.org/entity/Q158611,+44,American band,entity,music,musical group,cultural representative
4,http://www.wikidata.org/entity/Q280375,1 Monk Street,"building in Monmouth, Wales",entity,architecture,building,cultural exclusive



Validation Set:


,item,name,description,type,category,subcategory,label
0,http://www.wikidata.org/entity/Q15786,1. FC Nürnberg,"German sports club based in Nuremberg, Bavaria",entity,sports,sports club,cultural representative
1,http://www.wikidata.org/entity/Q268530,77 Records,UK record label,entity,music,record label,cultural exclusive
2,http://www.wikidata.org/entity/Q216153,A Bug's Life,1998 animated film directed by John Lasseter a...,entity,comics and anime,animated film,cultural representative
3,http://www.wikidata.org/entity/Q593,A Gang Story,2011 film by Olivier Marchal,entity,films,film,cultural exclusive
4,http://www.wikidata.org/entity/Q192185,Aaron Copland,"American composer, composition teacher, writer...",entity,performing arts,choreographer,cultural representative


In [4]:
df_train["label"].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic'], dtype=object)

In [5]:
print("HF dataset instance keys:", list(hf_dataset.keys()))
print("Train columns:", list(df_train.columns))
print("Val columns:", list(df_val.columns))

HF dataset instance keys: ['train', 'validation']
Train columns: ['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']
Val columns: ['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']


## LM-based Approach

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [7]:
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples["description"], truncation=True, padding=True)
    # Convert labels to numerical IDs using the label2id mapping
    tokenized_inputs["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_inputs

In [8]:
id2label = {
    0: "cultural agnostic",
    1: "cultural representative",
    2: "cultural exclusive",
}
label2id = {
    "cultural agnostic": 0,
    "cultural representative": 1,
    "cultural exclusive": 2,
}

In [9]:
def combine_fields(example):
    return {"text": f"{example['name']} [SEP] {example['description']}"}


hf_dataset = hf_dataset.map(combine_fields)

In [10]:
hf_dataset = hf_dataset.select_columns(["description", "label"])
tokenized_hf_dataset = hf_dataset.map(preprocess_function, batched=True)
tokenized_hf_dataset = tokenized_hf_dataset.select_columns(
    ["description", "input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [11]:
tokenized_hf_dataset

DatasetDict({
    train: Dataset({
        features: ['description', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6251
    })
    validation: Dataset({
        features: ['description', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", label2id=label2id, id2label=id2label
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
environ["WANDB_PROJECT"] = "mnlp"

In [21]:
from src.mnlp.utils.constants import LOG_DIR

training_args = TrainingArguments(
    output_dir=LOG_DIR / "distilbert-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_hf_dataset["train"],
    eval_dataset=tokenized_hf_dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.656638,0.683333
2,No log,0.659421,0.700000
3,No log,0.667912,0.723333
4,No log,0.738943,0.710000
5,No log,0.776110,0.720000
6,0.501000,0.823610,0.716667
7,0.501000,0.850700,0.713333
8,0.501000,0.893538,0.730000
9,0.501000,0.902090,0.706667
10,0.501000,0.911095,0.713333


TrainOutput(global_step=980, training_loss=0.3695426473812181, metrics={'train_runtime': 322.4163, 'train_samples_per_second': 193.88, 'train_steps_per_second': 3.04, 'total_flos': 1035085595201280.0, 'train_loss': 0.3695426473812181, 'epoch': 10.0})